In [1]:
import pandas as pd
import numpy as np
import gensim

C:\Users\Inês Valentim\Anaconda3\lib\site-packages\gensim\utils.py:840: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Inês Valentim\Anaconda3\lib\site-packages\gensim\utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [37]:
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

path_emails = 'hillary-clinton-emails/Emails.csv'

emails = pd.read_csv(path_emails, usecols=['ExtractedSubject','ExtractedBodyText','RawText'])

emails.fillna(' ', inplace=True)

emails['subject_body'] = emails['ExtractedSubject'] + ' ' + emails['ExtractedBodyText']

emails_raw = np.asarray(emails['subject_body'])

email_tokens = []

for email in emails_raw:
    email_tokens.append(regexp_tokenize(email, pattern='\w+'))

stop_words = set(stopwords.words('english'))

email_clean_tokens = []

# remove stopwords and numbers
for email in email_tokens:
    clean_tokens = [token for token in email if token.lower() not in stop_words and token.isdigit() == False]
    email_clean_tokens.append(clean_tokens)

stemmer = PorterStemmer()

email_clean = []

for email in email_clean_tokens:
    clean = [stemmer.stem(token.lower()) for token in email]
    email_clean.append(clean)

In [40]:
dictionary = gensim.corpora.Dictionary(email_clean)
corpus = [dictionary.doc2bow(email) for email in email_clean]

In [43]:
num_topics = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

for n in num_topics:
    lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=n, id2word=dictionary, passes=10)
    lda_model.print_topics(-1)